In [66]:
%pylab inline
from glob import glob
import numpy as np
from numpy import load
import pandas as pd
import tarfile
import os

Populating the interactive namespace from numpy and matplotlib


C:\Users\obaid\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['mean', 'std']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [67]:
import yaml
path = 'C:/Users/obaid/Desktop/bigdata/anon_images'
        
band_names=['Red','Green','Blue','NIR','SWIR1','SWIR2','TEMP1','NL']
print(path)

C:/Users/obaid/Desktop/bigdata/anon_images


In [68]:
files = 'C:/Users/obaid/Desktop/bigdata/anon_images'
files = glob(os.path.join(files, '*'))
len(files)

19669

In [69]:
#load table entries
train_table = r'https://raw.githubusercontent.com/SateeshKumar21/PovertyAnalysis/Vocareum/HW5/public_tables/train.csv__;!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62tQHdsHD$ ' 
#!ls -l $train_table

df=pd.read_csv(train_table,index_col=0)
df.index=df['filename']
df.head(3)

,filename,country,wealthpooled,urban,label,nl_mean
filename,,,,,,
image13724.npz,image13724.npz,6,-1.090052,False,0,-0.097684
image7407.npz,image7407.npz,6,-1.143002,False,0,-0.141589
image390.npz,image390.npz,6,1.056769,True,0,15.228898


In [70]:
len(df)

11288

In [71]:
import time

start_time=time.time()
channel_values_lis = []

floattypeerror=0
# change the range to load more data, I picked 5000 bc that creates ~4.2 GiBs to download
for n in range(11288):

    row=df.iloc[n,:]
    filename=row['filename']
    urban=row['urban']
    area ='urban' if urban else 'rural'
    try:
        M=load(path+'/'+filename)
    except:
        print (f'could not load image:{n}, path:'+path+"/"+filename)
        continue
    Image=M['x']
    if np.max(Image) > 65_536:
        print("above 16 byte size")
        channel_values_lis.append(Image)
        floattypeerror+=1
    elif np.max(Image) < -65_536:
        print("below 16 byte size")
        channel_values_lis.append(Image)
        floattypeerror+=1
    else:
        recast_arr = Image.astype(np.float16)
        channel_values_lis.append(recast_arr)
        

end_time= time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time,"seconds")
print("float type error:", print(floattypeerror))
print("length of file", len(channel_values_lis))
# uncomment to cave file
# np.save('channel_values_lis.npy', np.array(channel_values_lis))

Execution time: 113.27044248580933 seconds
0
float type error: None
length of file 11288


In [72]:
len(channel_values_lis)

11288

In [74]:
column_names=["Red", "Green", "Blue", "NIR", "SWIR1", "SWIR2", "TEMP1", "NL"]

In [75]:
#input image features into dataframe
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(channel_values_lis[n][column])
    df[column_names[column]] = lis

In [77]:
#mean
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(channel_values_lis[n][column].mean())
    df[column_names[column] + '_mean'] = lis

#std
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(np.std(channel_values_lis[n][column], dtype = np.float64))
    df[column_names[column] + '_std'] = lis

#median
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(np.median(channel_values_lis[n][column]))
    df[column_names[column] + '_median'] = lis

#bright outliers
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        mean_col = column_names[column] + '_mean'
        std_col = column_names[column] + '_std'
        mean = df.iloc[n][mean_col]
        std = df.iloc[n][std_col]
        lis.append(np.count_nonzero(channel_values_lis[n][column] > (mean + 1.5 * std)))
    df[column_names[column] + '_br_outlier'] = lis

#dark outlier
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        mean_col = column_names[column] + '_mean'
        std_col = column_names[column] + '_std'
        mean = df.iloc[n][mean_col]
        std = df.iloc[n][std_col]
        lis.append(np.count_nonzero(channel_values_lis[n][column] < (mean - 1.5 * std)))
    df[column_names[column] + '_dark_outlier'] = lis

In [80]:
urban_df = df[df['urban']==True]
rural_df = df[df['urban'] ==False]
urban_df_labels = urban_df[['wealthpooled', 'label']]
rural_df_labels = rural_df[['wealthpooled', 'label']]
urban_df_features = urban_df.drop(columns=["wealthpooled", "label"])
rural_df_features = rural_df.drop(columns=["wealthpooled", "label"])

print("urban_df_labels shape", urban_df_labels.shape)
print("rural_df_labels shape", rural_df_labels.shape)
print(" ")
print("urban_df_features shape", urban_df_features.shape)
print("rural_df_features shape", rural_df_features.shape)
######################
from sklearn.model_selection import train_test_split

#remove columns with the string values
numeric_columns = urban_df_features.select_dtypes(include=np.number).columns
urban_df_features_numeric = urban_df_features[numeric_columns]

#convert target variables
urban_df_labels.loc[:, 'wealthpooled'] = urban_df_labels['wealthpooled'].astype('float32')

#train-test split(urban data)
urban_X_train, urban_X_val, urban_y_train, urban_y_val = train_test_split(
    urban_df_features_numeric.values.astype('float32'),
    urban_df_labels['wealthpooled'].values,
    test_size=0.1,
    random_state=42
)

# #train-validation split (urban data)
#urban_X_train, urban_X_val, urban_y_train, urban_y_val = train_test_split(
#     urban_X_train,
#     urban_y_train,
#     test_size=0.11111,
#    random_state=42
#)

#remove columns with string values
numeric_columns = rural_df_features.select_dtypes(include=np.number).columns
rural_df_features_numeric = rural_df_features[numeric_columns]

#convert target variables
rural_df_labels.loc[:, 'wealthpooled'] = rural_df_labels['wealthpooled'].astype('float32')

#train-test split (rural data)
rural_X_train, rural_X_val, rural_y_train, rural_y_val = train_test_split(
    rural_df_features_numeric.values.astype('float32'),
    rural_df_labels['wealthpooled'].values,
    test_size=0.1,
    random_state=42
)

# #train-validation split for rural data
#rural_X_train, rural_X_val, rural_y_train, rural_y_val = train_test_split(
#     rural_X_train,
#     rural_y_train,
#     test_size=0.111111,
#     random_state=42
# )

print("Urban train shape:", urban_X_train.shape, urban_y_train.shape)
print("Urban validation shape:", urban_X_val.shape, urban_y_val.shape)
#print("Urban test shape:", urban_X_test.shape, urban_y_test.shape)
print("Rural train shape:", rural_X_train.shape, rural_y_train.shape)
print("Rural validation shape:", rural_X_val.shape, rural_y_val.shape)
#print("Rural test shape:", rural_X_test.shape, rural_y_test.shape)

urban_df_labels shape (4020, 2)
rural_df_labels shape (7268, 2)
 
urban_df_features shape (4020, 52)
rural_df_features shape (7268, 52)
Urban train shape: (3618, 42) (3618,)
Urban validation shape: (402, 42) (402,)
Rural train shape: (6541, 42) (6541,)
Rural validation shape: (727, 42) (727,)


C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\1429553098.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62itULFX1$   urban_df_labels.loc[:, 'wealthpooled'] = urban_df_labels['wealthpooled'].astype('float32')
C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\1429553098.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7h

In [81]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#normalize
scaler = StandardScaler()
urban_X_train_scaled = scaler.fit_transform(urban_X_train)
urban_X_val_scaled = scaler.transform(urban_X_val)
#urban_X_test_scaled = scaler.transform(urban_X_test)

rural_X_train_scaled = scaler.fit_transform(rural_X_train)
rural_X_val_scaled = scaler.fit_transform(rural_X_val)
#rural_X_test_scaled = scalar.transform(rural_X_test)

#convert labels
urban_y_train_binary = (urban_y_train > 1.2).astype(int)
urban_y_val_binary = (urban_y_val > 1.2).astype(int)
#urban_y_test_binary = (urban_y_test > 1.2).astype(int)

rural_y_train_binary = (rural_y_train > -0.5).astype(int)
rural_y_val_binary = (rural_y_val > -0.5).astype(int)
#rural_y_test_binary = (rural_y_test > -0.5).astype(int)

######make sure I change to rural or urban

urban_model = Sequential()
urban_model.add(Dense(64, activation='relu', input_shape=(42,)))
urban_model.add(Dense(32, activation='relu'))
urban_model.add(Dense(16, activation='relu'))
urban_model.add(Dense(1, activation='sigmoid'))
urban_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train, predict
urban_model.fit(urban_X_train_scaled, urban_y_train_binary, batch_size=64, epochs=30, validation_data=(urban_X_val_scaled, urban_y_val_binary))
#loss, accuracy = urban_model.evaluate(urban_X_test_scaled, urban_y_test_binary)
#print(f"Test Loss: {loss:.4f}")
#print(f"Test Accuracy: {accuracy:.4f}")
#print(predictions)

Epoch 1/30
57/57 [==============================] - 1s 4ms/step - loss: 0.6162 - accuracy: 0.6559 - val_loss: 0.5539 - val_accuracy: 0.7114
Epoch 2/30
57/57 [==============================] - 0s 2ms/step - loss: 0.5342 - accuracy: 0.7255 - val_loss: 0.5238 - val_accuracy: 0.7264
Epoch 3/30
57/57 [==============================] - 0s 2ms/step - loss: 0.5093 - accuracy: 0.7449 - val_loss: 0.5105 - val_accuracy: 0.7164
Epoch 4/30
57/57 [==============================] - 0s 2ms/step - loss: 0.4942 - accuracy: 0.7535 - val_loss: 0.5004 - val_accuracy: 0.7338
Epoch 5/30
57/57 [==============================] - 0s 2ms/step - loss: 0.4850 - accuracy: 0.7629 - val_loss: 0.4930 - val_accuracy: 0.7289
Epoch 6/30
57/57 [==============================] - 0s 2ms/step - loss: 0.4733 - accuracy: 0.7764 - val_loss: 0.4941 - val_accuracy: 0.7388
Epoch 7/30
57/57 [==============================] - 0s 2ms/step - loss: 0.4638 - accuracy: 0.7778 - val_loss: 0.4898 - val_accuracy: 0.7388
Epoch 8/30
57/57 [==

In [82]:
rural_model = Sequential()
rural_model.add(Dense(64, activation='relu', input_shape=(42,)))
rural_model.add(Dense(32, activation='relu'))
rural_model.add(Dense(16, activation='relu'))
rural_model.add(Dense(1, activation='sigmoid'))
rural_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train, predict
rural_model.fit(rural_X_train_scaled, rural_y_train_binary, batch_size=64, epochs=30, validation_data=(rural_X_val_scaled, rural_y_val_binary))
#loss, accuracy = rural_model.evaluate(rural_X_test_scaled, rural_y_test_binary)
#print(f"Test Loss: {loss:.4f}")
#print(f"Test Accuracy: {accuracy:.4f}")
#print(predictions)

Epoch 1/30
103/103 [==============================] - 1s 3ms/step - loss: 0.6266 - accuracy: 0.6534 - val_loss: 0.5911 - val_accuracy: 0.6933
Epoch 2/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5642 - accuracy: 0.7147 - val_loss: 0.5525 - val_accuracy: 0.7249
Epoch 3/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5391 - accuracy: 0.7312 - val_loss: 0.5494 - val_accuracy: 0.7235
Epoch 4/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5227 - accuracy: 0.7441 - val_loss: 0.5662 - val_accuracy: 0.7070
Epoch 5/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5111 - accuracy: 0.7500 - val_loss: 0.5432 - val_accuracy: 0.7331
Epoch 6/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5004 - accuracy: 0.7606 - val_loss: 0.5603 - val_accuracy: 0.7345
Epoch 7/30
103/103 [==============================] - 0s 2ms/step - loss: 0.4909 - accuracy: 0.7684 - val_loss: 0.5414 - val_accuracy: 0.7387
Epoch 

In [83]:
urban_df_features.columns

Index(['filename', 'country', 'urban', 'nl_mean', 'Red', 'Green', 'Blue',
       'NIR', 'SWIR1', 'SWIR2', 'TEMP1', 'NL', 'Red_mean', 'Green_mean',
       'Blue_mean', 'NIR_mean', 'SWIR1_mean', 'SWIR2_mean', 'TEMP1_mean',
       'NL_mean', 'Red_std', 'Green_std', 'Blue_std', 'NIR_std', 'SWIR1_std',
       'SWIR2_std', 'TEMP1_std', 'NL_std', 'Red_median', 'Green_median',
       'Blue_median', 'NIR_median', 'SWIR1_median', 'SWIR2_median',
       'TEMP1_median', 'NL_median', 'Red_br_outlier', 'Green_br_outlier',
       'Blue_br_outlier', 'NIR_br_outlier', 'SWIR1_br_outlier',
       'SWIR2_br_outlier', 'TEMP1_br_outlier', 'NL_br_outlier',
       'Red_dark_outlier', 'Green_dark_outlier', 'Blue_dark_outlier',
       'NIR_dark_outlier', 'SWIR1_dark_outlier', 'SWIR2_dark_outlier',
       'TEMP1_dark_outlier', 'NL_dark_outlier'],
      dtype='object')

In [84]:
#+without country
urban_df = df[df['urban']==True]
rural_df = df[df['urban'] ==False]
urban_df_labels = urban_df[['wealthpooled', 'label']]
rural_df_labels = rural_df[['wealthpooled', 'label']]

urban_df_features = urban_df.drop(columns=["wealthpooled", "label","country"])
rural_df_features = rural_df.drop(columns=["wealthpooled", "label","country"])

print("urban_df_labels shape", urban_df_labels.shape)
print("rural_df_labels shape", rural_df_labels.shape)
print(" ")
print("urban_df_features shape", urban_df_features.shape)
print("rural_df_features shape", rural_df_features.shape)
######################
from sklearn.model_selection import train_test_split

#remove columns with the string values
numeric_columns = urban_df_features.select_dtypes(include=np.number).columns
urban_df_features_numeric = urban_df_features[numeric_columns]

#convert target variables
urban_df_labels.loc[:, 'wealthpooled'] = urban_df_labels['wealthpooled'].astype('float32')

#train-test split(urban data)
urban_X_train, urban_X_val, urban_y_train, urban_y_val = train_test_split(
    urban_df_features_numeric.values.astype('float32'),
    urban_df_labels['wealthpooled'].values,
    test_size=0.1,
    random_state=42
)

# #train-validation split (urban data)
#urban_X_train, urban_X_val, urban_y_train, urban_y_val = train_test_split(
#     urban_X_train,
#     urban_y_train,
#     test_size=0.11111,
#    random_state=42
#)

#remove columns with string values
numeric_columns = rural_df_features.select_dtypes(include=np.number).columns
rural_df_features_numeric = rural_df_features[numeric_columns]

#convert target variables
rural_df_labels.loc[:, 'wealthpooled'] = rural_df_labels['wealthpooled'].astype('float32')

#train-test split (rural data)
rural_X_train, rural_X_val, rural_y_train, rural_y_val = train_test_split(
    rural_df_features_numeric.values.astype('float32'),
    rural_df_labels['wealthpooled'].values,
    test_size=0.1,
    random_state=42
)

# #train-validation split for rural data
#rural_X_train, rural_X_val, rural_y_train, rural_y_val = train_test_split(
#     rural_X_train,
#     rural_y_train,
#     test_size=0.111111,
#     random_state=42
# )

print("Urban train shape:", urban_X_train.shape, urban_y_train.shape)
print("Urban validation shape:", urban_X_val.shape, urban_y_val.shape)
#print("Urban test shape:", urban_X_test.shape, urban_y_test.shape)
print("Rural train shape:", rural_X_train.shape, rural_y_train.shape)
print("Rural validation shape:", rural_X_val.shape, rural_y_val.shape)
#print("Rural test shape:", rural_X_test.shape, rural_y_test.shape)

urban_df_labels shape (4020, 2)
rural_df_labels shape (7268, 2)
 
urban_df_features shape (4020, 51)
rural_df_features shape (7268, 51)
Urban train shape: (3618, 41) (3618,)
Urban validation shape: (402, 41) (402,)
Rural train shape: (6541, 41) (6541,)
Rural validation shape: (727, 41) (727,)


C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\440436543.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62itULFX1$   urban_df_labels.loc[:, 'wealthpooled'] = urban_df_labels['wealthpooled'].astype('float32')
C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\440436543.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvN

In [85]:
#without country

#normalize
scaler = StandardScaler()
urban_X_train_scaled = scaler.fit_transform(urban_X_train)
urban_X_val_scaled = scaler.transform(urban_X_val)
#urban_X_test_scaled = scaler.transform(urban_X_test)

rural_X_train_scaled = scaler.fit_transform(rural_X_train)
rural_X_val_scaled = scaler.fit_transform(rural_X_val)
#rural_X_test_scaled = scalar.transform(rural_X_test)

#convert labels
urban_y_train_binary = (urban_y_train > 1.2).astype(int)
urban_y_val_binary = (urban_y_val > 1.2).astype(int)
#urban_y_test_binary = (urban_y_test > 1.2).astype(int)

rural_y_train_binary = (rural_y_train > -0.5).astype(int)
rural_y_val_binary = (rural_y_val > -0.5).astype(int)
#rural_y_test_binary = (rural_y_test > -0.5).astype(int)

#####make sure I change to rural or urban

urban_model_cl = Sequential()
urban_model_cl.add(Dense(64, activation='relu', input_shape=(41,)))
urban_model_cl.add(Dense(32, activation='relu'))
urban_model_cl.add(Dense(16, activation='relu'))
urban_model_cl.add(Dense(1, activation='sigmoid'))
urban_model_cl.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train, predict
urban_model_cl.fit(urban_X_train_scaled, urban_y_train_binary, batch_size=64, epochs=30, validation_data=(urban_X_val_scaled, urban_y_val_binary))
#loss, accuracy = urban_model.evaluate(urban_X_test_scaled, urban_y_test_binary)
#print(f"Test Loss: {loss:.4f}")
#print(f"Test Accuracy: {accuracy:.4f}")
#print(predictions)

Epoch 1/30
57/57 [==============================] - 1s 3ms/step - loss: 0.5859 - accuracy: 0.6899 - val_loss: 0.5546 - val_accuracy: 0.7189
Epoch 2/30
57/57 [==============================] - 0s 2ms/step - loss: 0.5345 - accuracy: 0.7319 - val_loss: 0.5472 - val_accuracy: 0.6990
Epoch 3/30
57/57 [==============================] - 0s 1ms/step - loss: 0.5176 - accuracy: 0.7383 - val_loss: 0.5387 - val_accuracy: 0.7114
Epoch 4/30
57/57 [==============================] - 0s 1ms/step - loss: 0.5062 - accuracy: 0.7479 - val_loss: 0.5346 - val_accuracy: 0.7164
Epoch 5/30
57/57 [==============================] - 0s 2ms/step - loss: 0.4950 - accuracy: 0.7590 - val_loss: 0.5222 - val_accuracy: 0.7338
Epoch 6/30
57/57 [==============================] - 0s 1ms/step - loss: 0.4847 - accuracy: 0.7653 - val_loss: 0.5148 - val_accuracy: 0.7289
Epoch 7/30
57/57 [==============================] - 0s 1ms/step - loss: 0.4747 - accuracy: 0.7736 - val_loss: 0.5125 - val_accuracy: 0.7388
Epoch 8/30
57/57 [==

In [86]:
rural_model_cl = Sequential()
rural_model_cl.add(Dense(64, activation='relu', input_shape=(41,)))
rural_model_cl.add(Dense(32, activation='relu'))
rural_model_cl.add(Dense(16, activation='relu'))
rural_model_cl.add(Dense(1, activation='sigmoid'))
rural_model_cl.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train, predict
rural_model_cl.fit(rural_X_train_scaled, rural_y_train_binary, batch_size=64, epochs=30, validation_data=(rural_X_val_scaled, rural_y_val_binary))
#loss, accuracy = rural_model.evaluate(rural_X_test_scaled, rural_y_test_binary)
#print(f"Test Loss: {loss:.4f}")
#print(f"Test Accuracy: {accuracy:.4f}")
#print(predictions)

Epoch 1/30
103/103 [==============================] - 1s 3ms/step - loss: 0.6440 - accuracy: 0.6445 - val_loss: 0.6054 - val_accuracy: 0.6740
Epoch 2/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5877 - accuracy: 0.6981 - val_loss: 0.5845 - val_accuracy: 0.7070
Epoch 3/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5625 - accuracy: 0.7143 - val_loss: 0.5748 - val_accuracy: 0.7111
Epoch 4/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5427 - accuracy: 0.7265 - val_loss: 0.5761 - val_accuracy: 0.6988
Epoch 5/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5308 - accuracy: 0.7378 - val_loss: 0.5924 - val_accuracy: 0.7043
Epoch 6/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5191 - accuracy: 0.7442 - val_loss: 0.5640 - val_accuracy: 0.7153
Epoch 7/30
103/103 [==============================] - 0s 1ms/step - loss: 0.5085 - accuracy: 0.7505 - val_loss: 0.5714 - val_accuracy: 0.7290
Epoch 

In [87]:
#predictions = model.predict(urban_X_test_scaled)

In [88]:
df_country = pd.read_csv('https://urldefense.com/v3/__https://raw.githubusercontent.com/SateeshKumar21/PovertyAnalysis/main/HW5/public_tables/country_test_reduct.csv__;!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62l3mgyUz$ ', index_col=0)
df_rand = pd.read_csv('https://urldefense.com/v3/__https://raw.githubusercontent.com/SateeshKumar21/PovertyAnalysis/main/HW5/public_tables/random_test_reduct.csv',index_col=0__;!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62q4P8o0L$ )

In [89]:
##### country test
####replace df with df_test_country

import time

start_time=time.time()
channel_values_lis = []

floattypeerror=0
# change the range to load more data, I picked 5000 bc that creates ~4.2 GiBs to download
for n in range(len(df_country)):

    row=df_country.iloc[n,:]
    filename=row['filename']
    urban=row['urban']
    area ='urban' if urban else 'rural'
    try:
        M=load(path+'/'+filename)
    except:
        print (f'could not load image:{n}, path:'+path+"/"+filename)
        continue
    Image=M['x']
    if np.max(Image) > 65_536:
        print("above 16 byte size")
        channel_values_lis.append(Image)
        floattypeerror+=1
    elif np.max(Image) < -65_536:
        print("below 16 byte size")
        channel_values_lis.append(Image)
        floattypeerror+=1
    else:
        recast_arr = Image.astype(np.float16)
        channel_values_lis.append(recast_arr)
        

end_time= time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time,"seconds")
print("float type error:", print(floattypeerror))
print("length of file", len(channel_values_lis))
# uncomment to cave file
# np.save('channel_values_lis.npy', np.array(channel_values_lis))

#mean
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(channel_values_lis[n][column].mean())
    df_country[column_names[column] + '_mean'] = lis

#std
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(np.std(channel_values_lis[n][column], dtype = np.float64))
    df_country[column_names[column] + '_std'] = lis

#median
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(np.median(channel_values_lis[n][column]))
    df_country[column_names[column] + '_median'] = lis

#bright outliers
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        mean_col = column_names[column] + '_mean'
        std_col = column_names[column] + '_std'
        mean = df_country.iloc[n][mean_col]
        std = df_country.iloc[n][std_col]
        lis.append(np.count_nonzero(channel_values_lis[n][column] > (mean + 1.5 * std)))
    df_country[column_names[column] + '_br_outlier'] = lis

#dark outlier
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        mean_col = column_names[column] + '_mean'
        std_col = column_names[column] + '_std'
        mean = df_country.iloc[n][mean_col]
        std = df_country.iloc[n][std_col]
        lis.append(np.count_nonzero(channel_values_lis[n][column] < (mean - 1.5 * std)))
    df_country[column_names[column] + '_dark_outlier'] = lis

Execution time: 49.05574822425842 seconds
0
float type error: None
length of file 4653


In [90]:
df_country.columns

Index(['filename', 'country', 'urban', 'nl_mean', 'Red_mean', 'Green_mean',
       'Blue_mean', 'NIR_mean', 'SWIR1_mean', 'SWIR2_mean', 'TEMP1_mean',
       'NL_mean', 'Red_std', 'Green_std', 'Blue_std', 'NIR_std', 'SWIR1_std',
       'SWIR2_std', 'TEMP1_std', 'NL_std', 'Red_median', 'Green_median',
       'Blue_median', 'NIR_median', 'SWIR1_median', 'SWIR2_median',
       'TEMP1_median', 'NL_median', 'Red_br_outlier', 'Green_br_outlier',
       'Blue_br_outlier', 'NIR_br_outlier', 'SWIR1_br_outlier',
       'SWIR2_br_outlier', 'TEMP1_br_outlier', 'NL_br_outlier',
       'Red_dark_outlier', 'Green_dark_outlier', 'Blue_dark_outlier',
       'NIR_dark_outlier', 'SWIR1_dark_outlier', 'SWIR2_dark_outlier',
       'TEMP1_dark_outlier', 'NL_dark_outlier'],
      dtype='object')

In [98]:
urban_country_df = df_country[df_country['urban']==True]
rural_country_df = df_country[df_country['urban'] ==False]
rural_country_df.drop('country',axis=1,inplace=True)
urban_country_df.drop('country',axis=1,inplace=True)

print("urban_df_features shape", urban_country_df.shape)
print("rural_df_features shape", rural_country_df.shape)

numeric_columns = rural_country_df.select_dtypes(include=np.number).columns

rural_country = rural_country_df[numeric_columns]
urban_country = urban_country_df[numeric_columns]

scaler = StandardScaler()
urban_country_scaled = scaler.fit_transform(urban_country)
rural_country_scaled = scaler.fit_transform(rural_country)

urban_df_features shape (1448, 43)
rural_df_features shape (3205, 43)


C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\3718299548.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62itULFX1$   rural_country_df.drop('country',axis=1,inplace=True)
C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\3718299548.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62itULFX1$   urban_country_df.drop('country',axis=1,inplace=True)


In [97]:
rural_country.shape

(3205, 42)

In [93]:
##### country test
####replace df with df_test_rand

import time

start_time=time.time()
channel_values_lis = []

floattypeerror=0
# change the range to load more data, I picked 5000 bc that creates ~4.2 GiBs to download
for n in range(len(df_rand)):

    row=df_rand.iloc[n,:]
    filename=row['filename']
    urban=row['urban']
    area ='urban' if urban else 'rural'
    try:
        M=load(path+'/'+filename)
    except:
        print (f'could not load image:{n}, path:'+path+"/"+filename)
        continue
    Image=M['x']
    if np.max(Image) > 65_536:
        print("above 16 byte size")
        channel_values_lis.append(Image)
        floattypeerror+=1
    elif np.max(Image) < -65_536:
        print("below 16 byte size")
        channel_values_lis.append(Image)
        floattypeerror+=1
    else:
        recast_arr = Image.astype(np.float16)
        channel_values_lis.append(recast_arr)
        

end_time= time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time,"seconds")
print("float type error:", print(floattypeerror))
print("length of file", len(channel_values_lis))
# uncomment to cave file
# np.save('channel_values_lis.npy', np.array(channel_values_lis))

#mean
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(channel_values_lis[n][column].mean())
    df_rand[column_names[column] + '_mean'] = lis

#std
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(np.std(channel_values_lis[n][column], dtype = np.float64))
    df_rand[column_names[column] + '_std'] = lis

#median
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        lis.append(np.median(channel_values_lis[n][column]))
    df_rand[column_names[column] + '_median'] = lis

#bright outliers
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        mean_col = column_names[column] + '_mean'
        std_col = column_names[column] + '_std'
        mean = df_rand.iloc[n][mean_col]
        std = df_rand.iloc[n][std_col]
        lis.append(np.count_nonzero(channel_values_lis[n][column] > (mean + 1.5 * std)))
    df_rand[column_names[column] + '_br_outlier'] = lis

#dark outlier
for column in range(len(column_names)):
    lis = []
    for n in range(len(channel_values_lis)):
        mean_col = column_names[column] + '_mean'
        std_col = column_names[column] + '_std'
        mean = df_rand.iloc[n][mean_col]
        std = df_rand.iloc[n][std_col]
        lis.append(np.count_nonzero(channel_values_lis[n][column] < (mean - 1.5 * std)))
    df_rand[column_names[column] + '_dark_outlier'] = lis

Execution time: 47.25349473953247 seconds
0
float type error: None
length of file 4917


In [94]:
urban_rand_df = df_rand[df_rand['urban']==True]
rural_rand_df = df_rand[df_rand['urban'] ==False]

print("urban_df_features shape", urban_rand_df.shape)
print("rural_df_features shape", rural_rand_df.shape)

numeric_columns = rural_rand_df.select_dtypes(include=np.number).columns

rural_rand = rural_rand_df[numeric_columns]
urban_rand = urban_rand_df[numeric_columns]

scaler = StandardScaler()
urban_rand_scaled = scaler.fit_transform(urban_rand)
rural_rand_scaled = scaler.fit_transform(rural_rand)

urban_df_features shape (1707, 44)
rural_df_features shape (3210, 44)


In [95]:
urban_rand_df['preds_wo_abstention'] = urban_model.predict(urban_rand_scaled)
rural_rand_df['preds_wo_abstention'] = rural_model.predict(rural_rand_scaled)


df_rand_final = pd.concat([urban_rand_df, rural_rand_df])
df_rand_final.loc[df_rand_final['preds_wo_abstention']>=0.5,'pred_wo_abstention'] = int(1)

df_rand_final.loc[df_rand_final['preds_wo_abstention']<0.5,'pred_wo_abstention'] = int(-1)


df_rand_final = df_rand_final[['filename','urban','pred_wo_abstention']]
df_rand_final['urban'] = df_rand_final['urban'].replace({True: 1, False: 0})
#df_rand_final['pred_wo_abstention'] = df_rand_final['pred_wo_abstention'].replace({0:-1})

df_rand_final.to_csv('results.csv')

### need to concatenate then select correct columns

#df_rand_final = df

101/101 [==============================] - 0s 708us/step


C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\2378405921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62itULFX1$   urban_rand_df['preds_wo_abstention'] = urban_model.predict(urban_rand_scaled)
C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\2378405921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1i

In [100]:
urban_country_df['preds_with_abstention'] = urban_model_cl.predict(urban_country_scaled)
rural_country_df['preds_with_abstention'] = rural_model_cl.predict(rural_country_scaled)

df_country_final = pd.concat([urban_country_df,rural_country_df])
df_country_final.loc[df_country_final['preds_with_abstention']>=0.87,'pred_with_abstention'] = int(1)


df_country_final.loc[df_country_final['preds_with_abstention'] <=0.13,'pred_with_abstention'] = int(-1)

df_country_final.loc[((df_country_final['preds_with_abstention']<=0.87)&(df_country_final['preds_with_abstention']>=0.13)),'pred_with_abstention'] = int(0)

df_country_final = df_country_final[['filename','urban','pred_with_abstention']]
df_country_final['urban'] = df_country_final['urban'].replace({True:1, False: 0})

#df_country_final['pred_with_abstention'] = df_country_final['pred_with_abstention'].replace({0:-1})

df_country_final.to_csv('results_country.csv')

101/101 [==============================] - 0s 698us/step


C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\2106664636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvNsKmTyevDD2x1iJ_njKMCLQ_aOukCKCnrUvDbR4GD60DqBko1kwoXZpuW62itULFX1$   urban_country_df['preds_with_abstention'] = urban_model_cl.predict(urban_country_scaled)
C:\Users\obaid\AppData\Local\Temp\ipykernel_22788\2106664636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://urldefense.com/v3/__https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html*returning-a-view-versus-a-copy*5Cn__;IyU!!Mih3wA!B14CyWPRDqkcCvQ7hvNsK